<h1>Chapter 15. More about Type Annotations</h1>

<h2>Overloaded Signatures</h2>

The `@typing.overload` decorator in Python allows defining multiple type signatures for a single function, indicating it can accept different argument types and return types. It’s used for type checking, but only one actual implementation is provided.

In [1]:
import functools
import operator
from collections.abc import Iterable
from typing import TypeVar, Union, overload

# Define type variables to represent any type
T = TypeVar('T')
S = TypeVar('S')


# Overload function signatures for type checking
# Overload 1: Sum with no start value defaults to returning T or int
@overload
def sum(it: Iterable[T]) -> Union[T, int]: ...


# Overload 2: Sum with a specified start value returning T or S
@overload
def sum(it: Iterable[T], /, start: S) -> Union[T, S]: ...


def sum(it, /, start=0):
    return functools.reduce(operator.add, it, start)

<h3>Overload <code>max</code></h3>

In [2]:
from collections.abc import Callable, Iterable
from typing import Any, Protocol, TypeVar, Union, overload


class SupportsLessThan(Protocol):
    def __lt__(self, other: Any) -> bool: ...


T = TypeVar('T')
LT = TypeVar('LT')
DT = TypeVar('DT')

# Sentinel object for missing default argument
MISSING = object()
EMPTY_MSG = 'max() arg is an empty sequence'


@overload
def max(__arg1: LT, __arg2: LT, *args: LT, key: None = ...) -> LT: ...
@overload
def max(__arg1: T, __arg2: T, *args: T, key: Callable[[T], LT]) -> T: ...
@overload
def max(__iterable: Iterable[LT], *, key: None = ...) -> LT: ...
@overload
def max(__iterable: Iterable[T], *, key: Callable[[T], LT]) -> T: ...
@overload
def max(__iterable: Iterable[LT], *, key: None = ..., default: DT) -> Union[LT, DT]: ...
@overload
def max(
    __iterable: Iterable[T], *, key: Callable[[T], LT], default: DT
) -> Union[T, DT]: ...


def max(first, *args, key=None, default=MISSING):
    if args:
        series = args
        candidate = first
    else:
        series = iter(first)
        try:
            candidate = next(series)
        except StopIteration:
            if default is not MISSING:
                return default
            raise ValueError(EMPTY_MSG) from None
    if key is None:
        for current in series:
            if candidate < current:
                candidate = current
    else:
        candidate_key = key(candidate)
        for current in series:
            current_key = key(current)
            if candidate_key < current_key:
                candidate = current
                candidate_key = current_key
    return candidate

Arguments that implement `SupportLessThan` but without setting `key` and `default`

In [3]:
max(1, 2, -3)

2

In [4]:
max(['Go', 'Python', 'Rust'])

'Rust'

The `key` argument is set, the `default` argument is not

In [5]:
max(1, 2, -3, key=abs)

-3

In [6]:
max(['Go', 'Python', 'Rust'], key=len)

'Python'

The `default` argument is set, the `key` is not

In [7]:
max([1, 2, -3], default=0)

2

In [8]:
max([], default=None)  # returns None

The `key` and `default` arguments are set

In [9]:
max([1, 2, -3], key=abs, default=None)

-3

In [10]:
max([], key=abs, default=None)  # returns None

<h2>TypedDict</h2>

`TypedDict` is a class in Python’s `typing` module used to define dictionaries with a fixed set of keys, where each key is associated with a specific type. It allows for more precise type checking by specifying the types of individual dictionary entries, making it easier to work with structured data while catching type errors early.

In [11]:
from typing import TypedDict


class BookDict(TypedDict):
    isbn: str
    title: str
    authors: list[str]
    pagecount: int

In [12]:
pp = BookDict(
    isbn='0134757599',
    title='Refactoring, 2e',
    authors=['Martin Fowler', 'Kent Beck'],
    pagecount=256,
)

pp

{'isbn': '0134757599',
 'title': 'Refactoring, 2e',
 'authors': ['Martin Fowler', 'Kent Beck'],
 'pagecount': 256}

In [13]:
type(pp)

dict

In [14]:
try:
    pp.title
except AttributeError as e:
    print(e.__repr__())

AttributeError("'dict' object has no attribute 'title'")


In [15]:
pp['title']

'Refactoring, 2e'

In [16]:
BookDict.__annotations__

{'isbn': str, 'title': str, 'authors': list[str], 'pagecount': int}

Using `BookDict` in the function signature

In [17]:
AUTHOR_ELEMENT = '<AUTHOR>{}</AUTHOR>'


def to_xml(book: BookDict) -> str:
    elements: list[str] = []
    for key, value in book.items():
        if isinstance(value, list):
            elements.extend(AUTHOR_ELEMENT.format(n) for n in value)
        else:
            tag = key.upper()
            elements.append(f"<{tag}>{value}</{tag}>")
    xml = '\n\t'.join(elements)
    return f"<BOOK>\n\t{xml}\n</BOOK>"


print(to_xml(pp))

<BOOK>
	<ISBN>0134757599</ISBN>
	<TITLE>Refactoring, 2e</TITLE>
	<AUTHOR>Martin Fowler</AUTHOR>
	<AUTHOR>Kent Beck</AUTHOR>
	<PAGECOUNT>256</PAGECOUNT>
</BOOK>


Function `from_json` with variable annotations that parses a `JSON` formatted string of type `str` and returns `BookDict`

In [18]:
import json


def from_json(data: str) -> BookDict:
    whatever: BookDict = json.loads(data)
    return whatever

<h2>Reading Type Annotations During Execution</h2>

In [19]:
def clip(text: str, max_len: int = 80) -> str:
    """
    Return new 'str' clipped at last space before or after 'max_len'.
    Return full 'text' if no space found.
    """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:
        end = len(text)
    return text[:end].rstrip()

In [20]:
clip.__annotations__

{'text': str, 'max_len': int, 'return': str}

In [21]:
from __future__ import annotations

clip.__annotations__

{'text': str, 'max_len': int, 'return': str}

In [22]:
from typing import get_type_hints

get_type_hints(clip)

{'text': str, 'max_len': int, 'return': str}

The `_fields` class method in the `Checked` class returns a dictionary of the class's type annotations by using `get_type_hints()`, which retrieves the expected types of attributes.

In [23]:
class Checked:
    @classmethod
    def _fields(cls) -> dict[str, type]:
        return get_type_hints(cls)

<h2>Generalized Class Implementation</h2>

A generalized class in Python uses type variables or generics to handle multiple data types, ensuring flexibility, type safety, and reduced code duplication.

In [24]:
from typing import Generic, TypeVar

T = TypeVar('T')


class Box(Generic[T]):
    def __init__(self, item: T) -> None:
        self.item = item

    def get_item(self) -> T:
        return self.item

In [25]:
int_box = Box(123)
int_box.get_item()

123

In [26]:
str_box = Box('spam')
str_box.get_item()

'spam'

<h2>Variability</h2>

In Python, variability refers to how type relationships between parent and child types affect the behavior of generic classes or functions. This is relevant when using type annotations and generics to determine whether a type can be substituted with its subtypes or supertypes in different contexts.

<h3>Invariant</h3>

An invariant type does not allow substitution of a generic type with any other type, even if there is a parent-child relationship.

In [27]:
from typing import Generic, TypeVar


class Beverage:
    """Any Beverage."""


class Juice(Beverage):
    """Any fruit juice."""


class OrangeJuice(Juice):
    """Delicious juice from Brazilian oranges."""


T = TypeVar('T')


class BeverageDispenser(Generic[T]):
    """A dispenser parameterized on the beverage type."""

    def __init__(self, beverage: T) -> None:
        self.beverage = beverage

    def dispense(self) -> T:
        return self.beverage


def install(dispenser: BeverageDispenser[Juice]) -> None:
    """Install a fruit juice dispenser."""

The following code is acceptable

In [28]:
juice_dispenser = BeverageDispenser(Juice())
install(juice_dispenser)

The following code is not acceptable

In [29]:
juice_dispenser = BeverageDispenser(Beverage())
install(juice_dispenser)

<h3>Covariant</h3>

A covariant type allows a generic type to be substituted by its subtypes. This is typically used with immutable types.

In [30]:
from typing import Generic, TypeVar


class Beverage:
    """Any Beverage."""


class Juice(Beverage):
    """Any fruit juice."""


class OrangeJuice(Juice):
    """Delicious juice from Brazilian oranges."""


T_co = TypeVar('T_co', covariant=True)


class BeverageDispenser(Generic[T_co]):
    def __init__(self, beverage: T_co) -> None:
        self.beverage = beverage

    def dispense(self) -> T_co:
        return self.beverage


def install(dispenser: BeverageDispenser[Juice]) -> None:
    """Install a fruit juice dispenser."""

The following code is acceptable

In [31]:
juice_dispenser = BeverageDispenser(Juice())
install(juice_dispenser)

orange_juice_dispenser = BeverageDispenser(OrangeJuice())
install(orange_juice_dispenser)

The following code is not acceptable

In [32]:
beverage_dispenser = BeverageDispenser(Beverage())
install(beverage_dispenser)

<h3>Contravariant</h3>

A contravariant type allows a generic type to be substituted by its supertypes, meaning it is more general and accepts types further up the hierarchy.

In [33]:
from typing import Generic, TypeVar


class Refuse:
    """Any refuse."""


class Biodegradable(Refuse):
    """Biodegradable refuse."""


class Compostable(Biodegradable):
    """Compostable refuse."""


T_contra = TypeVar('T_contra', contravariant=True)


class TrashCan(Generic[T_contra]):
    def put(self, refuse: T_contra) -> None:
        """Store trash until dumped."""


def deploy(trash_can: TrashCan[Biodegradable]):
    """Deploy a trash can for biodegradable refuse."""

The following code is acceptable

In [34]:
bio_can: TrashCan[Biodegradable] = TrashCan()
deploy(bio_can)

trash_can: TrashCan[Refuse] = TrashCan()
deploy(trash_can)

The following code is not acceptable

In [35]:
compost_can: TrashCan[Compostable] = TrashCan()
deploy(compost_can)

<h2>Implementation of a Generic Static Protocol</h2>

A **generalized static protocol** defines flexible, generic interfaces using structural typing, ensuring objects meet method or attribute requirements with static type checking.

In [36]:
from typing import Protocol, TypeVar

T = TypeVar('T')


# Define a generalized protocol
class SupportsAdd(Protocol[T]):
    def __add__(self, other: T) -> T: ...


def add_two(a: SupportsAdd[T], b: T) -> T:
    return a + b

In [37]:
add_two(1, 2)

3

In [38]:
add_two('Hello, ', 'World!')

'Hello, World!'